# Packaging

## Packaging


Once we've made a working program, we'd like to be able to share it with others.

A good cross-platform build tool is the most important thing: you can always
have collaborators build from source.


## Laying out a project


When planning to package a project for distribution, defining a suitable
project layout is essential.




```
greetings                 <--- Parent directory for your project
├── greetings             <--- Directory containing the code for your package
│  ├── __init__.py        <--- Tells Python to treat the directory as a package
│  ├── command.py
│  └── greeter.py
├── LICENSE.md            <--- License to describe how others can use your package
├── pyproject.toml        <--- Configuration and metadata for your package
├── README.md             <--- Homepage to briefly describe how to install and use your package
└── tests                 <--- Tests for your package's functionality
   ├── __init__.py
   ├── fixtures
   └── test_greeter.py
```

## The package directory

### __init__.py files

In [ ]:
%%bash
touch greetings/greetings/test/__init__.py
touch greetings/greetings/__init__.py

## The package config file

### Using setuptools


To make python code into a package, we have to write a `setupfile`:







In [ ]:
%%writefile greetings/setup.py
from setuptools import setup, find_packages

setup(
    name="Greetings",
    version="0.1.0",
    packages=find_packages(exclude=["*test"]),
    entry_points={"console_scripts": ["greet = greetings.command:process"]},
)

### Specifying dependencies

We use the setup.py file to specify the packages we depend on:

```python
setup(
    name = "Greetings",
    version = "0.1.0",
    packages = find_packages(exclude=['*test']),
    install_requires = ['argparse', 'pyyaml']
)
```

## Installing the package




We can now install this code with


```
cd greetings
pip install .
```


And the package will be then available to use everywhere on the system.


In [ ]:
from greetings.greeter import greet

greet("James", "Hetherington")

### Developer Install


If you modify your source files, you would now find it appeared as if the program doesn't change.

That's because pip install **copies** the file.

(On my system to /Library/Python/2.7/site-packages/: this is operating
system dependent.)

If you want to install a package, but keep working on it, you can do


```
cd greetings
pip install -e .
```

### Installing from GitHub

If we have our code in a (public) git repo anyone can now install our package directly from the git URL:


```
pip install git+git://github.com/jamespjh/greeter
```

### Uploading to PyPI

We could now submit "greeter" to PyPI for approval, so everyone could `pip install` it.

## Make a command-line interface

In [ ]:
%%writefile greetings/greetings/command.py
from argparse import ArgumentParser
from .greeter import greet  # Note python 3 relative import


def process():
    parser = ArgumentParser(description="Generate appropriate greetings")

    parser.add_argument("--title", "-t")
    parser.add_argument("--polite", "-p", action="store_true")
    parser.add_argument("personal")
    parser.add_argument("family")

    arguments = parser.parse_args()

    print(
        greet(arguments.personal, arguments.family, arguments.title, arguments.polite)
    )


if __name__ == "__main__":
    process()

Specify entry point

In [ ]:
%%writefile greetings/setup.py

from setuptools import setup, find_packages

setup(
    name="Greetings",
    version="0.1.0",
    packages=find_packages(exclude=["*test"]),
    install_requires=["argparse", "pyyaml"],
    entry_points={"console_scripts": ["greet = greetings.command:process"]},
)


And the scripts are now available as command line commands:




In [ ]:
%%bash
greet --help

In [ ]:
%%bash
greet James Hetherington
greet --polite James Hetherington
greet James Hetherington --title Dr

## Write a readme file

e.g.:

In [ ]:
%%writefile greetings/README.md

Greetings!
==========

This is a very simple example package used as part of the Turing
[Research Software Engineering with Python](https://alan-turing-institute.github.io/rse-course) course.

Usage:
    
Invoke the tool with greet <FirstName> <Secondname>

## Write a license file

e.g.:

In [ ]:
%%writefile greetings/LICENSE.md

(C) The Alan Turing Institute 2021

This "greetings" example package is granted into the public domain.

## Write a citation file

e.g.:

In [ ]:
%%writefile greetings/CITATION.md

If you wish to refer to this course, please cite the URL
https://alan-turing-institute.github.io/rse-course

Portions of the material are taken from Software Carpentry
http://swcarpentry.org

You may well want to formalise this using the [codemeta.json](https://codemeta.github.io/) standard - this doesn't have wide adoption yet, but we recommend it.

## Write some unit tests


Separating the script from the logical module made this possible:







In [ ]:
%%writefile greetings/greetings/test/test_greeter.py
import yaml
import os
from ..greeter import greet


def test_greeter():
    with open(
        os.path.join(os.path.dirname(__file__), "fixtures", "samples.yaml")
    ) as fixtures_file:
        fixtures = yaml.safe_load(fixtures_file)
        for fixture in fixtures:
            answer = fixture.pop("answer")
            assert greet(**fixture) == answer




Add a fixtures file:







In [ ]:
%%writefile greetings/greetings/test/fixtures/samples.yaml
- personal: James
  family: Hetherington
  answer: "Hey, James Hetherington."
- personal: James
  family: Hetherington
  polite: True
  answer: "How do you do, James Hetherington."
- personal: James
  family: Hetherington
  title: Dr
  answer: "Hey, Dr James Hetherington."

In [ ]:
%%bash
pytest

## Documentation


Of course, there's more to do when taking code from a quick script and turning it into a proper module:







In [ ]:
%%writefile greetings/greetings/greeter.py
def greet(personal, family, title="", polite=False):
    """Generate a greeting string for a person.

    Parameters
    ----------
    personal: str
        A given name, such as Will or Jean-Luc
    family: str
        A family name, such as Riker or Picard
    title: str
        An optional title, such as Captain or Reverend
    polite: bool
        True for a formal greeting, False for informal.

    Returns
    -------
    string
        An appropriate greeting
    """

    greeting = "How do you do, " if polite else "Hey, "
    if title:
        greeting += title + " "

    greeting += personal + " " + family + "."
    return greeting

In [ ]:
import greetings

help(greetings.greeter.greet)




The documentation string explains how to use the function; don't worry about this for now, we'll consider
this next time.
